In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Mon Feb 10 10:30:09 2020

@author: hadi
"""
#lesion dataset ---- 8 classes

import keras
from keras.optimizers import SGD
from keras.layers import GlobalAveragePooling2D, Dropout, Dense
import tensorflow as tf
from keras.models import Model
from keras.models import model_from_json
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from keras import backend as K
from matplotlib import pyplot as plt
from sklearn.metrics import log_loss
from keras.utils import multi_gpu_model
from keras.utils import np_utils
import numpy as np
import sys
sys.setrecursionlimit(1000)
from keras.preprocessing.image import ImageDataGenerator
from timeit import default_timer as timer    
from keras_preprocessing.image import ImageDataGenerator
from numpy import linspace
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="2"
from tensorflow.python.client import device_lib
device_lib.list_local_devices()# Check if we're using a GPU device
from sklearn.metrics import classification_report, confusion_matrix
import efficientnet.tfkeras
from tensorflow.keras.models import load_model
import efficientnet.keras as efn 
import efficientnet.tfkeras
from tensorflow.keras.models import load_model

Using TensorFlow backend.


In [7]:
channel = 3; bs = 16 #batch_size
nb_epoch = 10;
img_rows, img_cols = 224, 224; # Resolution of inputs
dirc = '/home/hadi/python/SparseFool/ordered/CFAR_10/dataset/cifar/raw/';#skin lesion directory




datagen = ImageDataGenerator(rescale=1./255.,rotation_range = 90,horizontal_flip=True,vertical_flip = False)
train_generator = datagen.flow_from_directory(
        directory = dirc + '/train',
        target_size=(img_rows, img_cols), color_mode='rgb',
        classes=['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'],
        class_mode='categorical', batch_size=bs, shuffle=True, seed=42, subset=None, interpolation='bicubic' )

valid_generator = datagen.flow_from_directory(
        directory=dirc + '/val',
        target_size=(img_rows, img_cols), color_mode='rgb',
        classes=['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'],
        class_mode='categorical',batch_size=bs, shuffle=True, seed=42,subset=None, interpolation='bicubic')

datagenTest = ImageDataGenerator(rescale=1./255.,rotation_range = 0,horizontal_flip=False,vertical_flip = False)


# test_generator = datagenTest.flow_from_directory(
#         directory=dirc + '/test',
#         target_size=(img_rows, img_cols), color_mode='rgb',
#         classes=['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'],
#         class_mode='categorical',batch_size=bs, shuffle=True, seed=42,subset=None, interpolation='bicubic')

# test_generator_sparsified_50 = datagenTest.flow_from_directory(
#         directory='/home/hadi/python/SparseFool/ordered/CFAR_10/dataset/cifar/sparsified/50Perc/test',
#         target_size=(img_rows, img_cols), color_mode='rgb',
#         classes=['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'],
#         class_mode='categorical',batch_size=bs, shuffle=False, seed=42,subset=None, interpolation='bicubic')


# test_generator_sparsified_60 = datagenTest.flow_from_directory(
#         directory='/home/hadi/python/SparseFool/ordered/CFAR_10/dataset/cifar/sparsified/60Perc/test',
#         target_size=(img_rows, img_cols), color_mode='rgb',
#         classes=['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'],
#         class_mode='categorical',batch_size=bs, shuffle=False, seed=42,subset=None, interpolation='bicubic')



# test_generator_sparsified_70 = datagenTest.flow_from_directory(
#         directory='/home/hadi/python/SparseFool/ordered/CFAR_10/dataset/cifar/sparsified/70Perc/test',
#         target_size=(img_rows, img_cols), color_mode='rgb',
#         classes=['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'],
#         class_mode='categorical',batch_size=bs, shuffle=False, seed=42,subset=None, interpolation='bicubic')

# test_generator_sparsified_80 = datagenTest.flow_from_directory(
#         directory='/home/hadi/python/SparseFool/ordered/CFAR_10/dataset/cifar/sparsified/80Perc/test',
#         target_size=(img_rows, img_cols), color_mode='rgb',
#         classes=['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'],
#         class_mode='categorical',batch_size=bs, shuffle=False, seed=42,subset=None, interpolation='bicubic')

# ##################################Low Frequency
# test_generator_sparsified_LoF_50 = datagenTest.flow_from_directory(
#         directory='/home/hadi/python/SparseFool/ordered/CFAR_10/dataset/cifar/sparsified/LoF/50Perc/test',
#         target_size=(img_rows, img_cols), color_mode='rgb',
#         classes=['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'],
#         class_mode='categorical',batch_size=bs, shuffle=False, seed=42,subset=None, interpolation='bicubic')

# test_generator_sparsified_LoF_60 = datagenTest.flow_from_directory(
#         directory='/home/hadi/python/SparseFool/ordered/CFAR_10/dataset/cifar/sparsified/LoF/60Perc/test',
#         target_size=(img_rows, img_cols), color_mode='rgb',
#         classes=['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'],
#         class_mode='categorical',batch_size=bs, shuffle=False, seed=42,subset=None, interpolation='bicubic')

# test_generator_sparsified_LoF_70 = datagenTest.flow_from_directory(
#         directory='/home/hadi/python/SparseFool/ordered/CFAR_10/dataset/cifar/sparsified/LoF/70Perc/test',
#         target_size=(img_rows, img_cols), color_mode='rgb',
#         classes=['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'],
#         class_mode='categorical',batch_size=bs, shuffle=False, seed=42,subset=None, interpolation='bicubic')

# test_generator_sparsified_LoF_80 = datagenTest.flow_from_directory(
#         directory='/home/hadi/python/SparseFool/ordered/CFAR_10/dataset/cifar/sparsified/LoF/80Perc/test',
#         target_size=(img_rows, img_cols), color_mode='rgb',
#         classes=['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'],
#         class_mode='categorical',batch_size=bs, shuffle=False, seed=42,subset=None, interpolation='bicubic')
########################High Frequency

test_generator_sparsified_HiF_50 = datagenTest.flow_from_directory(
        directory='/home/hadi/python/SparseFool/ordered/CFAR_10/dataset/cifar/sparsified/HiF/50Perc/test',
        target_size=(img_rows, img_cols), color_mode='rgb',
        classes=['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'],
        class_mode='categorical',batch_size=bs, shuffle=False, seed=42,subset=None, interpolation='bicubic')

test_generator_sparsified_HiF_60 = datagenTest.flow_from_directory(
        directory='/home/hadi/python/SparseFool/ordered/CFAR_10/dataset/cifar/sparsified/HiF/60Perc/test',
        target_size=(img_rows, img_cols), color_mode='rgb',
        classes=['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'],
        class_mode='categorical',batch_size=bs, shuffle=False, seed=42,subset=None, interpolation='bicubic')

test_generator_sparsified_HiF_70 = datagenTest.flow_from_directory(
        directory='/home/hadi/python/SparseFool/ordered/CFAR_10/dataset/cifar/sparsified/HiF/70Perc/test',
        target_size=(img_rows, img_cols), color_mode='rgb',
        classes=['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'],
        class_mode='categorical',batch_size=bs, shuffle=False, seed=42,subset=None, interpolation='bicubic')

test_generator_sparsified_HiF_80 = datagenTest.flow_from_directory(
        directory='/home/hadi/python/SparseFool/ordered/CFAR_10/dataset/cifar/sparsified/HiF/80Perc/test',
        target_size=(img_rows, img_cols), color_mode='rgb',
        classes=['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'],
        class_mode='categorical',batch_size=bs, shuffle=False, seed=42,subset=None, interpolation='bicubic')

Found 50000 images belonging to 10 classes.
Found 0 images belonging to 10 classes.
Found 10000 images belonging to 10 classes.
Found 10000 images belonging to 10 classes.
Found 10000 images belonging to 10 classes.
Found 10000 images belonging to 10 classes.


In [3]:


base_model = keras.applications.resnet.ResNet50(include_top=False, weights="imagenet",input_shape=(img_rows, img_cols, 3));  # or weights='noisy-student'
CLASSES = 10
x = base_model.output
x = GlobalAveragePooling2D(name='avg_pool')(x)
x = Dropout(0.4)(x)
predictions = Dense(CLASSES, activation='softmax')(x)
parallel_model = Model(inputs=base_model.input, outputs=predictions)

sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
parallel_model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])



history2 = parallel_model.fit(train_generator,
                steps_per_epoch=int(1000),verbose=1,epochs=nb_epoch,validation_data=valid_generator)


print(test_generator.class_indices)
score = parallel_model.evaluate(test_generator, verbose=1)
print("%s: %.2f%%" % (parallel_model.metrics_names[1], score[1]*100))

W0228 06:25:17.646185 140608547145472 deprecation_wrapper.py:119] From /home/hadi/anaconda3/envs/venv1/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4070: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



94773248/94765736 [==============================] - 2s 0us/step


W0228 06:25:29.794682 140608547145472 deprecation_wrapper.py:119] From /home/hadi/anaconda3/envs/venv1/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:422: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.



Epoch 1/10
1000/1000 [==============================] - 157s 157ms/step - loss: 1.0791 - accuracy: 0.6301
Epoch 2/10
1000/1000 [==============================] - 152s 152ms/step - loss: 0.6448 - accuracy: 0.7777
Epoch 3/10
1000/1000 [==============================] - 149s 149ms/step - loss: 0.5466 - accuracy: 0.8112
Epoch 4/10
1000/1000 [==============================] - 152s 152ms/step - loss: 0.4603 - accuracy: 0.8416
Epoch 5/10
1000/1000 [==============================] - 151s 151ms/step - loss: 0.4188 - accuracy: 0.8562
Epoch 6/10
1000/1000 [==============================] - 151s 151ms/step - loss: 0.3994 - accuracy: 0.8618
Epoch 7/10
1000/1000 [==============================] - 153s 153ms/step - loss: 0.3561 - accuracy: 0.8778
Epoch 8/10
1000/1000 [==============================] - 152s 152ms/step - loss: 0.3426 - accuracy: 0.8841
Epoch 9/10
1000/1000 [==============================] - 152s 152ms/step - loss: 0.3279 - accuracy: 0.8873
Epoch 10/10
1000/1000 [=======================

In [4]:
model_path = '/home/hadi/python/SparseFool/ordered/CFAR_10/effectOfSparsity/ResNet50/saved_model/'
parallel_model.save(model_path + 'ResNet50_CIFAR10_raw_10.h5')
print('model saved')


print(test_generator.class_indices)
score = parallel_model.evaluate(test_generator, verbose=1)
print("%s: %.2f%%" % (parallel_model.metrics_names[1], score[1]*100))

{'airplane': 0, 'automobile': 1, 'bird': 2, 'cat': 3, 'deer': 4, 'dog': 5, 'frog': 6, 'horse': 7, 'ship': 8, 'truck': 9}
625/625 [==============================] - 30s 48ms/step
accuracy: 91.37%


In [3]:
model_path = '/home/hadi/python/SparseFool/ordered/CFAR_10/effectOfSparsity/ResNet50/saved_model/'
load_mode = True
if load_mode:
    parallel_model = keras.models.load_model(model_path + 'ResNet50_CIFAR10_raw_10.h5')
    sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
    parallel_model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
    print('model is loaded')

W0228 23:44:25.731170 140644134160128 deprecation_wrapper.py:119] From /home/hadi/anaconda3/envs/venv1/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4070: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0228 23:44:36.123939 140644134160128 deprecation_wrapper.py:119] From /home/hadi/anaconda3/envs/venv1/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:422: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.



model is loaded


In [5]:
score = parallel_model.evaluate(test_generator, verbose=1)
print('raw: ',"%s: %.2f%%" % (parallel_model.metrics_names[1], score[1]*100))

print('---------------------------------------------------------------------')

score = parallel_model.evaluate(test_generator_sparsified_50, verbose=1)
print('50%: ', "%s: %.2f%%" % (parallel_model.metrics_names[1], score[1]*100))

print('---------------------------------------------------------------------')

score = parallel_model.evaluate(test_generator_sparsified_60, verbose=1)
print('60%: ', "%s: %.2f%%" % (parallel_model.metrics_names[1], score[1]*100))

print('---------------------------------------------------------------------')

score = parallel_model.evaluate(test_generator_sparsified_70, verbose=1)
print('70%: ', "%s: %.2f%%" % (parallel_model.metrics_names[1], score[1]*100))

print('---------------------------------------------------------------------')

score = parallel_model.evaluate(test_generator_sparsified_80, verbose=1)
print('80%: ', "%s: %.2f%%" % (parallel_model.metrics_names[1], score[1]*100))

625/625 [==============================] - 30s 48ms/step
raw:  accuracy: 91.37%
---------------------------------------------------------------------
625/625 [==============================] - 30s 48ms/step
50%:  accuracy: 90.73%
---------------------------------------------------------------------
625/625 [==============================] - 30s 48ms/step
60%:  accuracy: 89.77%
---------------------------------------------------------------------
625/625 [==============================] - 30s 48ms/step
70%:  accuracy: 87.59%
---------------------------------------------------------------------
625/625 [==============================] - 29s 47ms/step
80%:  accuracy: 81.70%


In [5]:
score = parallel_model.evaluate(test_generator_sparsified_LoF_50, verbose=1)
print('50%: ', "%s: %.2f%%" % (parallel_model.metrics_names[1], score[1]*100))
print('---------------------------------------------------------------------')

score = parallel_model.evaluate(test_generator_sparsified_LoF_60, verbose=1)
print('60%: ', "%s: %.2f%%" % (parallel_model.metrics_names[1], score[1]*100))
print('---------------------------------------------------------------------')


score = parallel_model.evaluate(test_generator_sparsified_LoF_70, verbose=1)
print('70%: ', "%s: %.2f%%" % (parallel_model.metrics_names[1], score[1]*100))
print('---------------------------------------------------------------------')

score = parallel_model.evaluate(test_generator_sparsified_LoF_80, verbose=1)
print('80%: ', "%s: %.2f%%" % (parallel_model.metrics_names[1], score[1]*100))
print('---------------------------------------------------------------------')


625/625 [==============================] - 28s 45ms/step
50%:  accuracy: 79.30%
---------------------------------------------------------------------
625/625 [==============================] - 29s 46ms/step
60%:  accuracy: 76.67%
---------------------------------------------------------------------
625/625 [==============================] - 29s 46ms/step
70%:  accuracy: 73.29%
---------------------------------------------------------------------
625/625 [==============================] - 29s 46ms/step
80%:  accuracy: 51.69%
---------------------------------------------------------------------


In [8]:
score = parallel_model.evaluate(test_generator_sparsified_HiF_50, verbose=1)
print('50%: ', "%s: %.2f%%" % (parallel_model.metrics_names[1], score[1]*100))
print('---------------------------------------------------------------------')

score = parallel_model.evaluate(test_generator_sparsified_HiF_60, verbose=1)
print('60%: ', "%s: %.2f%%" % (parallel_model.metrics_names[1], score[1]*100))
print('---------------------------------------------------------------------')


score = parallel_model.evaluate(test_generator_sparsified_HiF_70, verbose=1)
print('70%: ', "%s: %.2f%%" % (parallel_model.metrics_names[1], score[1]*100))
print('---------------------------------------------------------------------')

score = parallel_model.evaluate(test_generator_sparsified_HiF_80, verbose=1)
print('80%: ', "%s: %.2f%%" % (parallel_model.metrics_names[1], score[1]*100))
print('---------------------------------------------------------------------')


625/625 [==============================] - 29s 46ms/step
50%:  accuracy: 20.89%
---------------------------------------------------------------------
625/625 [==============================] - 29s 46ms/step
60%:  accuracy: 15.38%
---------------------------------------------------------------------
625/625 [==============================] - 29s 46ms/step
70%:  accuracy: 16.13%
---------------------------------------------------------------------
625/625 [==============================] - 28s 45ms/step
80%:  accuracy: 11.14%
---------------------------------------------------------------------
